In [1]:
import os
import os.path as osp
import mylib.file_utils as fu
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
from mmcv.runner import CheckpointLoader
import torch
from torch import nn

In [3]:
a=(3, 4, 6, 3)
sum(a)

16

In [21]:
x = torch.rand(size=(2,2,2,2))
training = True
drop_prob = 0.5
if drop_prob == 0. or not training:
    output = x
else:
    keep_prob = 1 - drop_prob
    # handle tensors with different dimensions, not just 4D tensors.
    shape = (x.shape[0], ) + (1, ) * (x.ndim - 1)
    random_tensor = keep_prob + torch.rand(
        shape, dtype=x.dtype, device=x.device)
    output = x.div(keep_prob) * random_tensor.floor()
    
print(f'shape: {shape}\nrandom tensor: {random_tensor}\nfloor: {random_tensor.floor()}\nori: {x}\noutput: {output}')

shape: (2, 1, 1, 1)
random tensor: tensor([[[[0.7363]]],


        [[[0.5601]]]])
floor: tensor([[[[0.]]],


        [[[0.]]]])
ori: tensor([[[[0.9559, 0.5000],
          [0.3690, 0.2141]],

         [[0.7701, 0.3394],
          [0.8624, 0.3983]]],


        [[[0.4529, 0.4521],
          [0.8011, 0.6279]],

         [[0.8408, 0.3655],
          [0.0844, 0.8882]]]])
output: tensor([[[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]]])


In [4]:
a=torch.Tensor([0, 1])
b = torch.Tensor([2,0])
min(a.max(), b.max())

(tensor([0, 1, 2, 3]), tensor([1, 2, 3, 2]))

In [3]:
path = r'work_dirs/pbyol_r50_sn6_sar_pro_ep200/20210927_094326/epoch_200.pth'
ckpt = torch.load(path)

In [9]:
online = ckpt['state_dict']
print(len(online))
for k, v in online.items():
    print(k)

1017
online_net.0.stem.0.weight
online_net.0.stem.1.weight
online_net.0.stem.1.bias
online_net.0.stem.1.running_mean
online_net.0.stem.1.running_var
online_net.0.stem.1.num_batches_tracked
online_net.0.stem.3.weight
online_net.0.stem.4.weight
online_net.0.stem.4.bias
online_net.0.stem.4.running_mean
online_net.0.stem.4.running_var
online_net.0.stem.4.num_batches_tracked
online_net.0.stem.6.weight
online_net.0.stem.7.weight
online_net.0.stem.7.bias
online_net.0.stem.7.running_mean
online_net.0.stem.7.running_var
online_net.0.stem.7.num_batches_tracked
online_net.0.layer1.0.conv1.weight
online_net.0.layer1.0.bn1.weight
online_net.0.layer1.0.bn1.bias
online_net.0.layer1.0.bn1.running_mean
online_net.0.layer1.0.bn1.running_var
online_net.0.layer1.0.bn1.num_batches_tracked
online_net.0.layer1.0.conv2.weight
online_net.0.layer1.0.bn2.weight
online_net.0.layer1.0.bn2.bias
online_net.0.layer1.0.bn2.running_mean
online_net.0.layer1.0.bn2.running_var
online_net.0.layer1.0.bn2.num_batches_tracked

In [7]:
a='SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190823162315_20190823162606_tile_7868.tif'
b = a.split('.')
c = b[0].split('.')
b, b[0], c

(['SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190823162315_20190823162606_tile_7868',
  'tif'],
 'SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190823162315_20190823162606_tile_7868',
 ['SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190823162315_20190823162606_tile_7868'])

In [2]:
a = np.arange(12).reshape(2,2,3)
b = np.array([[0, 1], [1, 0]]).astype(np.bool8)
c = a[b]
# d = np.tile(b, (2, 1, 1))
# d = np.repeat(b[..., None], 2, axis=-1)
# e = a[d]
# print(a.shape, b.shape, c.shape, d.shape, e.shape)
print(f'a: {a.shape}\n{a}\nb: {b.shape}\n{b}\nc: {c.shape}\n{c}')
print('d: {d.shape}\n{d}')
# print(f'e: {e.shape}\n{e}')
print(f'expanded b:{b[..., None].shape}\n{b[..., None]}')

a: (2, 2, 3)
[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]
b: (2, 2)
[[False  True]
 [ True False]]
c: (2, 3)
[[3 4 5]
 [6 7 8]]
d: {d.shape}
{d}
expanded b:(2, 2, 1)
[[[False]
  [ True]]

 [[ True]
  [False]]]


In [5]:
img_path = r'/data/csl/SN6_full/SAR-PRO/SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190822092149_20190822092450_tile_1697.tif'
mask_path = r'/data/csl/SN6_sup/slic_mask/SN6_Train_AOI_11_Rotterdam_SAR-Intensity_20190822092149_20190822092450_tile_1697.png'
img = Image.open(img_path)
mask = Image.open(mask_path)

In [18]:
nimg = np.asarray(img)
nmask = np.asarray(mask)
nimg.shape, nimg.max(), nimg.min(), nmask.shape, nmask.max(), nmask.min()

((900, 900, 3), 224, 0, (900, 900), 92, 0)

In [15]:
def cat_pil_images(images, axis):
    arrays = [np.asarray(img) for img in images]
    arrays = [arr if arr.ndim==3 else arr[..., None] for arr in arrays]
    array = np.concatenate(arrays, axis=axis)
    return Image.fromarray(array)
mix = cat_pil_images((img, mask), axis=-1)


In [17]:
nmix = np.asarray(mix)
nmix.shape, nmix.max(), nmix.min()

((900, 900, 4), 224, 0)